<a href="https://colab.research.google.com/github/RANA1804/Introduction_to_machine_learning/blob/main/01_Intro_to_geemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to geemap
## Installing the Updated Version of geemap
Install geemap version 0.29.3 or a later release to enable the automatic authentication feature, as this functionality is only supported in these versions.To install other Python packages, you can use the pip install package_name command. To update a package, you can use pip install --upgrade package_name or pip install -U package_name

In [ ]:
%pip install --upgrade geemap

## Import required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ee
import geemap

## Innitialize a map
When initializing a Map object, you may be prompted for authorization. If this occurs, you can obtain the required authorization token by visiting the provided link.

Certainly! When working with the Map object in a Python environment, you can play with various parameters to customize the map display.

In [ ]:
Map = geemap.Map() #geemap is a librarry & Map is a Class
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Change the map
Map = geemap.Map(height="400pt", width= "100%", basemap = "Esri.WorldImagery")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# We can use this way to Change the basemap layer to 'Esri World Imagery'
Map.add_basemap(basemap="Esri.WorldImagery")

Esri.WorldImagery has been already added before.


## Showing the name of all Basemaps

In [ ]:
basemaps = geemap.basemaps
for basemap in basemaps:
  print(basemap)

## Working with Feature Collection
In Google Earth Engine, a Feature Collection is a type of data structure that represents a collection of vector features. These features could represent points, lines, polygons, or a combination of these geometries. Feature Collections are fundamental for working with spatial data and conducting geospatial analyses in Earth Engine.



In [ ]:
# Set up a new map object
Map = geemap.Map()
Map

In [ ]:
# Imput the shapefile (in this case World Administrative Boundary)
world = ee.FeatureCollection("users/geonextgis/World_Administrative_Boundaries")

world_style = {
    "fillColor": "yellow", # First 6 digit for RGB color last 2 digit for itransparency value
    "color": "black", # This is the color of the border
    "width": 1 # This is the strock of the border
}
Map.addLayer(world.style(**world_style), {}, "World Administrative Boundary") # Kwargs - dictionary

## Filtering Feature Collection
Filtering a Feature Collection in Google Earth Engine involves selecting a subset of features based on specific criteria, such as spatial, attribute, or temporal conditions. This process is essential for isolating relevant data for analysis. Here are the key aspects of filtering a Feature Collection:

### Attribute Filtering:

Attribute filtering involves selecting features based on their attribute values, such as properties or characteristics.
The filter function is often used in combination with ee.Filter to define attribute-based conditions.


In [ ]:
# Filter all the European countries
european_countries = world.filter(ee.Filter.eq("continent", "Europe"))
european_countries_style = {
    "fillColor": "green",
    "color": "black",
    "width": 1
}
Map.addLayer(european_countries.style(**european_countries_style), {}, "European Countries")
Map.centerObject(european_countries, 3)

### Spatial Filtering

* Spatial filtering involves selecting features based on their geographic  location or proximity to a specified region.
* Common spatial filters include geometry, intersects, bounds, and distance, allowing users to focus on features within a defined area or at a certain distance from a given point.





In [ ]:
# Read 'Grided Population Data' from NASA SEDAC
gpw = ee.FeatureCollection("projects/sat-io/open-datasets/sedac/gpw-v4-admin-unit-center-points-population-estimates-rev11"); # When we use the .style
# function then the vector file converted as a image

# Filter the Feature collection with only European contries (this is the sptiala filtering)
# Filter the points where population estimates is more than 500000 in 2020 (this is attribute filtering)
gpw_european = gpw.filterBounds(european_countries)

gpw_european_gt5lakhs_20 = gpw_european.filter(ee.Filter.gt("UN_2020_E", 500000))
gpw_europeanan_gt5lakhs_20_style = {
    "fillColor": "C70039",
    "color" : "red",
    "width" : 1,
    "pointSize" : 5
}
Map.addLayer(gpw_european_gt5lakhs_20.style(**gpw_europeanan_gt5lakhs_20_style), {}, "Global Population Estimates > 10 Lakhs (Asia)")
### Now the whole vector file and the points are a image ###

🤔 Note: In geemap, the addLayer function is designed to visualize data on the map by adding a layer. However, it's important to note that this function always returns an Image object, not a Feature Collection or individual Feature when using with style function

## Download the Filtered Feature Collection

In [ ]:
task = ee.batch.Export.table.toDrive(collection = gpw_european_gt5lakhs_20,
                                     description = "GPW_European_Pop_Est_2020",
                                     folder = "GEE",
                                     fileNamePrefix = "GPW_European_Pop_Est_2020",
                                     fileFormat = "SHP")
# Export the filtered feature collection
#task.start()

## **Working with Image Collection**
In Google Earth Engine (GEE), an Image Collection is a fundamental data structure used to represent a group or sequence of images. These images can be satellite observations, remotely sensed data, or any other raster data that can be organized over time or space.



In [ ]:
map = geemap.Map(height = "300pt", width = "80%")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
marker = map.draw_last_feature
marker

## Import the LANDSAT 9 Level 2 tyer 2 Image Collection

In [ ]:
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
l9

##Filtering Image Collection

*   We can filter Image Collections based on various criteria, such as date range, spatial extent, or metadata properties.


*   To optimize the workflow, it is advisable to follow a specific order when filtering an Image Collection. The recommended sequence includes filtering by boundary first, followed by dates, and then metadata properties.
*   This approach helps reduce computational load and speeds up the execution of operations.

In [ ]:
l9FilteredImage = l9.filterBounds(marker.geometry())\
                    .filterDate("2022-01-01", "2022-12-31")\
                    .filterMetadata("CLOUD_COVER", "Less_than", 10)\
                    .first()
rgb_Vis = {
    "min": 8500,
    "max": 17000,
    "bands": ["SR_B4", "SR_B3", "SR_B2"]
}

sfcc_vis = {
    "min": 8000,
    "max": 18000,
    "bands": ["SR_B5", "SR_B4", "SR_B3"]
}

Map.addLayer(l9FilteredImage, rgb_Vis, "RGB Image")
Map.addLayer(l9FilteredImage, sfcc_vis, "SFCC Image")
Map.centerObject(l9FilteredImage, 8)

In [ ]:
l9FilteredImage

In [ ]:
Map

## Store the Meadata of the ImageCollection

In [ ]:
# Store the Metadata properties name as a list
Image_property_names = l9FilteredImage.propertyNames()
Image_property_names

In [ ]:
type(Image_property_names)

ee.ee_list.List

In [ ]:
# Print the metadata properties information
image_property = l9FilteredImage.toDictionary(Image_property_names)
image_property

In [ ]:
type(image_property)

ee.dictionary.Dictionary

## Convert the GEE server-side data to client-side data

🤔 Note: getInfo() is a method in GEE API that allows users to retrieve the values of Earth Engine objects and transfer them from the server-side to the client-side. In the context of GEE, computations often occur on the server-side, which means that the actual data and results reside on Google's servers. To access and work with this information in your local environment, we use getInfo().

In [ ]:
image_property = l9FilteredImage.toDictionary(Image_property_names).getInfo()
image_property

In [ ]:
type(image_property)